In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from wordcloud import WordCloud
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

trainData=pd.read_csv("/train.csv")
testData = pd.read_csv("/test.csv")

In [ ]:
trainData.isnull().sum()

id                          0
belongs_to_collection    2396
budget                      0
genres                      7
homepage                 2054
imdb_id                     0
original_language           0
original_title              0
overview                    8
popularity                  0
poster_path                 1
production_companies      156
production_countries       55
release_date                0
runtime                     2
spoken_languages           20
status                      0
tagline                   597
title                       0
Keywords                  276
cast                       13
crew                       16
revenue                     0
dtype: int64

In [ ]:
testData.isnull().sum()

id                          0
belongs_to_collection    3521
budget                      0
genres                     16
homepage                 2978
imdb_id                     0
original_language           0
original_title              0
overview                   14
popularity                  0
poster_path                 1
production_companies      258
production_countries      102
release_date                1
runtime                     4
spoken_languages           42
status                      2
tagline                   863
title                       3
Keywords                  393
cast                       13
crew                       22
dtype: int64

In [ ]:
def get_dict(s):
    try:
        x = eval(s)
    except:
        x = {}
    return x

In [ ]:
print( len(trainData[trainData['budget'] < 10000] ) )

835


In [ ]:
trainData = trainData[trainData['budget'] >= 100000]

In [ ]:
MEAN = trainData['budget'].mean()

In [ ]:
trainData['budget']= trainData['budget'].apply(lambda x : x+MEAN if x<100000  else x)

In [ ]:
trainData['log_budget'] = np.log1p(trainData['budget'])
testData['log_budget'] = np.log1p(testData['budget'])

# Generos

In [ ]:
j = 0
trainData['genre_Count'] = 0
for i in trainData['genres']:
    d = get_dict(i)
    if d != {}:
        
        trainData['genre_Count'][j] = len(d)
    else:
        trainData['genre_Count'][j] = 0 
    j += 1

j = 0
testData['genre_Count'] = 0
for i in testData['genres']:
    d = get_dict(i)
    if d != {}:
        
        testData['genre_Count'][j] = len(d)
    else:
        testData['genre_Count'][j] = 0 
    j += 1
trainData.drop(['genres'], axis=1, inplace=True)
testData.drop(['genres'], axis=1, inplace=True)
trainData.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: Setting

,id,belongs_to_collection,budget,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,...,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue,log_budget,genre_Count
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651,16.454568,1
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435,17.504390,4
2,3,NaN,3300000,http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000,15.009433,1
3,4,NaN,1200000,http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000,13.997833,2
5,6,NaN,8000000,NaN,tt0093743,en,Pinocchio and the Emperor of the Night,"Pinocchio and his friends, a glow worm and a m...",0.743274,/6IDqA1D2NBIVhzEEaMMRL28iBrq.jpg,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Pinocchio and the Emperor of the Night,NaN,"[{'cast_id': 6, 'character': 'Pinocchio (voice...","[{'credit_id': '52fe46f49251416c9106558b', 'de...",3261638,15.894952,2


# Eliminando columnas 

In [ ]:
# Elimiando HomePage
trainData.drop(['homepage'], axis=1, inplace=True)
testData.drop(['homepage'], axis=1, inplace=True)
# Elimiando imdb_id
trainData.drop(['imdb_id'], axis=1, inplace=True)
testData.drop(['imdb_id'], axis=1, inplace=True)
# Elimiando original_title
trainData.drop(['original_title','overview'], axis=1, inplace=True)
testData.drop(['original_title','overview'], axis=1, inplace=True)
# Elimiando poster_path
trainData.drop(['poster_path'], axis=1, inplace=True)
testData.drop(['poster_path'], axis=1, inplace=True)
# Elimiando spoken_languages


# Lenguaje Original

Utilizaremos las peliculas con un lenguaje original en ingles = EN , Ya que es el mayor porcentaje que hay en el dataset

In [ ]:
#Train Data
for i in trainData['original_language']:
    if(i=='en'):
        trainData['Has_En']=1
    else:
        trainData['Has_En']=0

#Test Data
for i in testData['original_language']:
    if(i=='en'):
        testData['Has_En']=1
    else:
        testData['Has_En']=0


trainData.drop(['original_language'], axis=1, inplace=True)
testData.drop(['original_language'], axis=1, inplace=True)

In [ ]:
for i,e in enumerate(trainData["production_companies"][:5]):
    print(i,e)

num_production_companies=pd.DataFrame(columns=['num_production_companies'])
trainData=pd.concat([trainData,num_production_companies],axis=1)
testData=pd.concat([testData,num_production_companies],axis=1)

j=0
for i in trainData["production_companies"]:
    d=get_dict(i)
    if len(d)!=0:
        trainData['num_production_companies'][j]=len(d)
    else:
        trainData['num_production_companies'][j]=np.NaN
    j+=1
j=0
for i in testData["production_companies"]:
    d=get_dict(i)
    if len(d)!=0:
        testData['num_production_companies'][j]=len(d)
    else:
        testData['num_production_companies'][j]=np.NaN
    j+=1

0 [{'name': 'Paramount Pictures', 'id': 4}, {'name': 'United Artists', 'id': 60}, {'name': 'Metro-Goldwyn-Mayer (MGM)', 'id': 8411}]
1 [{'name': 'Walt Disney Pictures', 'id': 2}]
2 [{'name': 'Bold Films', 'id': 2266}, {'name': 'Blumhouse Productions', 'id': 3172}, {'name': 'Right of Way Films', 'id': 32157}]
3 nan
4 nan


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarni

In [ ]:
trainData.drop(['production_companies'], axis=1, inplace=True)
testData.drop(['production_companies'], axis=1, inplace=True)


In [ ]:
trainData["num_production_companies"].isnull().sum()


668

In [ ]:
trainData['num_production_companies']=trainData['num_production_companies'].fillna(trainData['num_production_companies'].mean()).round()
testData['num_production_companies']=testData['num_production_companies'].fillna(testData['num_production_companies'].mean()).round()

Paises de Produccion 

In [ ]:
for i,e in enumerate(trainData["production_countries"][:5]):
    print(i,e)

0 [{'iso_3166_1': 'US', 'name': 'United States of America'}]
1 [{'iso_3166_1': 'US', 'name': 'United States of America'}]
2 [{'iso_3166_1': 'US', 'name': 'United States of America'}]
3 [{'iso_3166_1': 'IN', 'name': 'India'}]
4 nan


In [ ]:
prodect_countries=pd.DataFrame(columns=["production_country"])
trainData=pd.concat([trainData,prodect_countries],axis=1)
testData=pd.concat([testData,prodect_countries],axis=1)

In [ ]:
j = 0
for i in trainData['production_countries']:
    d = get_dict(i)
    if d != {}:
        if len(d) > 1:
            countires = []
            for k in range(len(d)):
                countires.append(d[k]['name'])
            if 'United States of America' in countires:
                trainData['production_country'][j] = 'United States of America'
        else:
            trainData['production_country'][j] = d[0]['name']
    else:
        trainData['production_country'][j] = np.NaN
    j += 1
    
j = 0
for i in testData['production_countries']:
    d = get_dict(i)
    if d != {}:
        if len(d) > 1:
            countires = []
            for k in range(len(d)):
                countires.append(d[k]['name'])
            if 'United States of America' in countires:
                testData['production_country'][j] = 'United States of America'
        else:
            testData['production_country'][j] = d[0]['name']
    else:
        testData['production_country'][j] = np.NaN
    j += 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-p

In [ ]:
is_use_preduction=pd.DataFrame(columns=["is_use_preduction"])
trainData=pd.concat([trainData,is_use_preduction],axis=1)
testData=pd.concat([testData,is_use_preduction],axis=1)

In [ ]:
j=0
for i in trainData["production_country"]:
    if i=='United States of America':
        trainData['is_use_preduction'][j]=1
    elif str(i)=='NaN':
        trainData['is_use_preduction'][j]=np.NaN
    else:
        trainData['is_use_preduction'][j]=0
    j+=1
j=0
for i in testData["production_country"]:
    if i=='United States of America':
        testData['is_use_preduction'][j]=1
    elif str(i)=='NaN':
        testData['is_use_preduction'][j]=np.NaN
    else:
        testData['is_use_preduction'][j]=0
    j+=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_laun

In [ ]:
trainData.drop(['production_countries','production_country'], axis=1, inplace=True)
testData.drop(['production_countries','production_country'], axis=1, inplace=True)


In [ ]:
trainData['is_use_preduction'].isnull().sum()

618

runtime

In [ ]:
for i,e in enumerate(trainData['runtime'][:5]):
    print(i,e)

trainData['runtime'].isnull().sum()

0 93.0
1 113.0
2 105.0
3 122.0
4 83.0


1

In [ ]:
trainData['runtime']=trainData['runtime'].fillna(trainData['runtime'].mean()).round()
testData['runtime']=testData['runtime'].fillna(testData['runtime'].mean()).round()
trainData['runtime'].isnull().sum()

0

In [ ]:
for i,e in enumerate(trainData['spoken_languages'][:5]):
    print(i,e)

0 [{'iso_639_1': 'en', 'name': 'English'}]
1 [{'iso_639_1': 'en', 'name': 'English'}]
2 [{'iso_639_1': 'en', 'name': 'English'}]
3 [{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'hi', 'name': 'हिन्दी'}]
4 [{'iso_639_1': 'en', 'name': 'English'}]


In [ ]:
number_of_spoken_languages = pd.DataFrame(columns=['number_of_spoken_languages'])
trainData = pd.concat([trainData, number_of_spoken_languages], axis=1)
testData = pd.concat([testData, number_of_spoken_languages], axis=1)

In [ ]:
j = 0
for i in trainData['spoken_languages']:
    d = get_dict(i)
    if d != {}:
        trainData['number_of_spoken_languages'][j] = len(d)
    else:
        trainData['number_of_spoken_languages'][j] = np.NaN
    j += 1
    
j = 0
for i in testData['spoken_languages']:
    d = get_dict(i)
    if d != {}:
        testData['number_of_spoken_languages'][j] = len(d)
    else:
        testData['number_of_spoken_languages'][j] = np.NaN
    j += 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A 

In [ ]:
trainData["number_of_spoken_languages"].isnull().sum()

621

In [ ]:
trainData['number_of_spoken_languages']=trainData['number_of_spoken_languages'].fillna(trainData['number_of_spoken_languages'].mean()).round()
testData['number_of_spoken_languages']=testData['number_of_spoken_languages'].fillna(testData['number_of_spoken_languages'].mean()).round()

In [ ]:
trainData.drop(['spoken_languages'], axis=1, inplace=True)
testData.drop(['spoken_languages'], axis=1, inplace=True)

# Estatus

In [ ]:
trainData['status'].value_counts()

testData['status'].value_counts()

Released           4389
Post Production       5
Rumored               2
Name: status, dtype: int64

In [ ]:
Status=pd.DataFrame(columns=["Status"])
trainData=pd.concat([trainData,Status],axis=1)
testData=pd.concat([testData,Status],axis=1)

In [ ]:
j=0
for i in trainData["status"]:
    if i=='Released':
        trainData['Status'][j]=1
    elif str(i)=='NaN':
        trainData['Status'][j]=np.NaN
    else:
        trainData['Status'][j]=0
    j+=1
j=0
for i in testData["status"]:
    if i=='Released':
        testData['Status'][j]=1
    elif str(i)=='NaN':
        testData['Status'][j]=np.NaN
    else:
        testData['Status'][j]=0
    j+=1
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_laun

In [ ]:
trainData.drop(['status'], axis=1, inplace=True)
testData.drop(['status'], axis=1, inplace=True)

In [ ]:
trainData.drop(['Status'], axis=1, inplace=True)
testData.drop(['Status'], axis=1, inplace=True)

In [ ]:
trainData.drop(['title','release_date','tagline'], axis=1, inplace=True)
testData.drop(['title','release_date','tagline'], axis=1, inplace=True)

In [ ]:
#palabras claves
trainData.drop(['Keywords'], axis=1, inplace=True)
testData.drop(['Keywords'], axis=1, inplace=True)

In [ ]:
size_of_cast = pd.DataFrame(columns=['size_of_cast'])
trainData = pd.concat([trainData, size_of_cast], axis=1)
testData = pd.concat([testData, size_of_cast], axis=1)

In [ ]:
j = 0
for i in trainData['cast']:
    d = get_dict(i)
    if d != {}:
        trainData['size_of_cast'][j] = len(d)
    else:
        trainData['size_of_cast'][j] = np.NaN
    j += 1
    
j = 0
for i in testData['cast']:
    d = get_dict(i)
    if d != {}:
        testData['size_of_cast'][j] = len(d)
    else:
        testData['size_of_cast'][j] = np.NaN
    j += 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A 

In [ ]:
trainData.drop(['cast'], axis=1, inplace=True)
testData.drop(['cast'], axis=1, inplace=True)

In [ ]:
trainData["size_of_cast"].isnull().sum()

627

In [ ]:
trainData['size_of_cast']=trainData['size_of_cast'].fillna(trainData['size_of_cast'].mean()).round()
testData['size_of_cast']=testData['size_of_cast'].fillna(testData['size_of_cast'].mean()).round()

In [ ]:
trainData.drop(['crew'], axis=1, inplace=True)
testData.drop(['crew'], axis=1, inplace=True)

In [ ]:
trainData.drop(['belongs_to_collection'], axis=1, inplace=True)
testData.drop(['belongs_to_collection'], axis=1, inplace=True)

In [ ]:
testData.head()

,id,budget,popularity,runtime,log_budget,genre_Count,Has_En,num_production_companies,is_use_preduction,number_of_spoken_languages,size_of_cast
0,3001,0,3.851534,90.0,0.000000,4,0,3.0,1,2.0,7.0
1,3002,88000,3.559789,65.0,11.385103,2,0,1.0,1,1.0,10.0
2,3003,0,8.085194,100.0,0.000000,2,0,3.0,1,1.0,9.0
3,3004,6800000,8.596012,130.0,15.732433,3,0,3.0,0,3.0,23.0
4,3005,2000000,3.217680,92.0,14.508658,2,0,3.0,1,1.0,4.0


In [ ]:
trainData.head()


,id,budget,popularity,runtime,revenue,log_budget,genre_Count,Has_En,num_production_companies,is_use_preduction,number_of_spoken_languages,size_of_cast
0,1,14000000,6.575393,93.0,12314651,16.454568,1,1,3.0,1,1.0,24.0
1,2,40000000,8.248895,113.0,95149435,17.504390,4,1,1.0,1,1.0,20.0
2,3,3300000,64.299990,105.0,13092000,15.009433,1,1,3.0,1,1.0,51.0
3,4,1200000,3.174936,122.0,16000000,13.997833,2,1,3.0,0,2.0,7.0
5,6,8000000,0.743274,83.0,3261638,15.894952,2,1,2.0,1,1.0,14.0


In [ ]:
y=trainData.revenue

In [ ]:
features=['log_budget','popularity',"runtime"]

In [ ]:
train_x=trainData[features]

In [ ]:
Rf_model=RandomForestRegressor(random_state=1)
Rf_model.fit(train_x,y)
Rf_prediction=Rf_model.predict(train_x)
mean_absolute_error(Rf_prediction,y)

20202943.660097625

In [ ]:
r2_score(Rf_prediction,y)

0.9304375285237262

Prediccion usando test data

In [ ]:
test_id=testData["id"]
test_id.shape
test_x=testData[features]
Rf_predictions=Rf_model.predict(test_x)

In [ ]:
submission = pd.DataFrame({'id': test_id, 'revenue':Rf_predictions})
submission = submission[['id', 'revenue']]

submission.to_csv("submission.csv", index=False)

In [ ]:
print(Rf_predictions)

[1.13221754e+07 9.84796000e+06 4.16255027e+06 ... 7.54750459e+07
 1.84780713e+08 1.35027201e+07]
